# Лабораторная работа №5

## Подготовка данных

In [5]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [6]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

In [7]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

In [8]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

In [9]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

In [11]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Дерево решений

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [24]:
des_tree_classifier = DecisionTreeClassifier()
des_tree_params ={
    "criterion": ['gini','entropy','log_loss'],
    'splitter': ['best','random'],
    'max_depth':range(1,100),
}
des_tree_grid = GridSearchCV(des_tree_classifier, des_tree_params, cv=2, n_jobs=-1,verbose=3)
des_tree_grid.fit(X_train, y_train)
best_des_tree_params = des_tree_grid.best_params_

Fitting 2 folds for each of 594 candidates, totalling 1188 fits


In [25]:
best_des_tree_model = DecisionTreeClassifier(**best_des_tree_params)
best_des_tree_model.fit(X_train, y_train)
des_tree_predicted = best_des_tree_model.predict(X_test)
print('Used params:',best_des_tree_params)
print('Evaluation:\n', metrics.classification_report(y_test, des_tree_predicted,digits=5))

Used params: {'criterion': 'entropy', 'max_depth': 5, 'splitter': 'best'}
Evaluation:
               precision    recall  f1-score   support

       False    0.86402   0.93721   0.89913      9078
        True    0.68404   0.47960   0.56386      2573

    accuracy                        0.83615     11651
   macro avg    0.77403   0.70840   0.73149     11651
weighted avg    0.82427   0.83615   0.82509     11651



### Случайный лес

In [27]:
import numpy as np

from sklearn.ensemble import RandomForestClassifier

In [28]:
rand_forest_classifier = RandomForestClassifier()
rand_forest_params ={
    'n_estimators': np.geomspace(1,1e2,10,dtype=int),
    'criterion': ['gini','entropy','log_loss'],
    'max_depth':range(1,100)
}
rand_forest_grid = GridSearchCV(rand_forest_classifier, rand_forest_params, cv=2, n_jobs=-1,verbose=3)
rand_forest_grid.fit(X_train, y_train)
rand_forest_best_params = rand_forest_grid.best_params_

Fitting 2 folds for each of 2970 candidates, totalling 5940 fits


In [29]:
rand_forest_best_model = RandomForestClassifier(**rand_forest_best_params)
rand_forest_best_model.fit(X_train, y_train)
rand_forest_predicted = rand_forest_best_model.predict(X_test)
print('Used params:',rand_forest_best_params)
print('Evaluation:\n', metrics.classification_report(y_test, rand_forest_predicted,digits=5))

Used params: {'criterion': 'log_loss', 'max_depth': 66, 'n_estimators': 100}
Evaluation:
               precision    recall  f1-score   support

       False    0.86772   0.94878   0.90644      9078
        True    0.73043   0.48970   0.58632      2573

    accuracy                        0.84740     11651
   macro avg    0.79908   0.71924   0.74638     11651
weighted avg    0.83740   0.84740   0.83575     11651

